# Example Seldon Core Deployments using Helm
<img src="images/deploy-graph.png" alt="predictor with canary" title="ml graph"/>

## Prerequistes
You will need
 - [Git clone of Seldon Core](https://github.com/SeldonIO/seldon-core)
 - A running Kubernetes cluster with kubectl authenticated
 - [python grpc tools](https://grpc.io/docs/quickstart/python.html)
 - [Helm client](https://helm.sh/)

### Creating a Kubernetes Cluster

Follow the [Kubernetes documentation to create a cluster](https://kubernetes.io/docs/setup/).

Once created ensure ```kubectl``` is authenticated against the running cluster.

# Setup

In [ ]:
!kubectl create namespace seldon

In [ ]:
!kubectl config set-context $(kubectl config current-context) --namespace=seldon

In [ ]:
!kubectl create clusterrolebinding kube-system-cluster-admin --clusterrole=cluster-admin --serviceaccount=kube-system:default

# Install Helm

In [ ]:
!kubectl -n kube-system create sa tiller
!kubectl create clusterrolebinding tiller --clusterrole cluster-admin --serviceaccount=kube-system:tiller
!helm init --service-account tiller

In [ ]:
!kubectl rollout status deploy/tiller-deploy -n kube-system

## Start seldon-core

In [ ]:
!helm install ../../helm-charts/seldon-core-crd --name seldon-core-crd --set usage_metrics.enabled=true

In [ ]:
!helm install ../../helm-charts/seldon-core --name seldon-core --namespace seldon  --set ambassador.enabled=true

In [ ]:
!kubectl rollout status deploy/seldon-core-seldon-cluster-manager
!kubectl rollout status deploy/seldon-core-seldon-apiserver
!kubectl rollout status deploy/seldon-core-ambassador

## Set up REST and gRPC methods

**Ensure you port forward ambassador and API gateway**:

```
kubectl port-forward $(kubectl get pods -n seldon -l service=ambassador -o jsonpath='{.items[0].metadata.name}') -n seldon 8003:8080
```

```
kubectl port-forward $(kubectl get pods -n seldon -l app=seldon-apiserver-container-app -o jsonpath='{.items[0].metadata.name}') -n seldon 8002:8080
```

```
kubectl port-forward $(kubectl get pods -n seldon -l app=seldon-apiserver-container-app -o jsonpath='{.items[0].metadata.name}') -n seldon 8004:5000
```

## Serve Single Model

In [ ]:
!helm install ../../helm-charts/seldon-single-model --name mymodel --set model.image.name=seldonio/mock_classifier_rest:1.2 --set oauth.key=oauth-key --set oauth.secret=oauth-secret

In [ ]:
!helm template ../../helm-charts/seldon-single-model --name mymodel --set oauth.key=oauth-key --set oauth.secret=oauth-secret | pygmentize -l json

In [ ]:
!kubectl rollout status deploy/mymodel-mymodel-951ca8b

### Get predictions

In [2]:
from seldon_core.seldon_client import SeldonClient
sc = SeldonClient(deployment_name="seldon-model",namespace="default",oauth_key="oauth-key",oauth_secret="oauth-secret")

#### REST Request

In [3]:
p = sc.predict(gateway="seldon",transport="rest")
print(p)

Success:True message:
Request:
data {
  tensor {
    shape: 1
    shape: 1
    values: 0.6063688924809905
  }
}

Response:
meta {
  puid: "t9cmosba6ik44h5h8runvu44bv"
  requestPath {
    key: "classifier"
    value: "seldonio/mock_classifier:1.0"
  }
}
data {
  names: "proba"
  tensor {
    shape: 1
    shape: 1
    values: 0.09027368720811113
  }
}



In [ ]:
p = sc.predict(gateway="ambassador",transport="rest")
print(p)

#### gRPC Request

In [ ]:
p = sc.predict(gateway="ambassador",transport="grpc")
print(p)

In [ ]:
p = sc.predict(gateway="seldon",transport="grpc")
print(p)

## Test Feedback

In [ ]:
p = sc.predict(gateway="seldon",transport="rest")
f = sc.feedback(p.request,p.response,1.0,gateway="seldon",transport="rest")
print(f)

In [ ]:
p = sc.predict(gateway="ambassador",transport="rest")
f = sc.feedback(p.request,p.response,1.0,gateway="ambassador",transport="rest")
print(f)

### gRPC

In [ ]:
p = sc.predict(gateway="ambassador",transport="grpc")
f = sc.feedback(p.request,p.response,1.0,gateway="ambassador",transport="grpc")
print(f)

In [ ]:
p = sc.predict(gateway="seldon",transport="grpc")
f = sc.feedback(p.request,p.response,1.0,gateway="seldon",transport="grpc")
print(f)

In [ ]:
!helm delete mymodel --purge

## Serve AB Test

In [ ]:
!helm install ../../helm-charts/seldon-abtest --name myabtest --set oauth.key=oauth-key --set oauth.secret=oauth-secret

In [ ]:
!kubectl rollout status deploy/myabtest-abtest-41de5b8
!kubectl rollout status deploy/myabtest-abtest-df66c5c

### Get predictions

In [ ]:
sc = SeldonClient(deployment_name="myabtest",namespace="seldon",oauth_key="oauth-key",oauth_secret="oauth-secret")

In [ ]:
r = sc.predict(gateway="seldon",transport="rest")
print(r)

In [ ]:
r = sc.predict(gateway="ambassador",transport="rest")
print(r)

#### gRPC Request

In [ ]:
r = sc.predict(gateway="ambassador",transport="grpc")
print(r)

In [ ]:
r = sc.predict(gateway="seldon",transport="grpc")
print(r)

In [ ]:
!helm delete myabtest --purge

## Serve Multi-Armed Bandit

In [ ]:
!helm install ../../helm-charts/seldon-mab --name mymab --set modela.image.name=seldonio/mock_classifier_rest:1.2 --set modelb.image.name=seldonio/mock_classifier_rest:1.2 --set oauth.key=oauth-key --set oauth.secret=oauth-secret

In [ ]:
!kubectl rollout status deploy/mymab-abtest-295a20e
!kubectl rollout status deploy/mymab-abtest-b8038b2
!kubectl rollout status deploy/mymab-abtest-5724b8a

### Get predictions

In [ ]:
sc = SeldonClient(deployment_name="mymab",namespace="seldon",oauth_key="oauth-key",oauth_secret="oauth-secret")

In [ ]:
r = sc.predict(gateway="seldon",transport="rest")
print(r)

In [ ]:
r = sc.predict(gateway="ambassador",transport="rest")
print(r)

#### gRPC Request

In [ ]:
r = sc.predict(gateway="ambassador",transport="grpc")
print(r)

In [ ]:
r = sc.predict(gateway="seldon",transport="grpc")
print(r)

In [ ]:
!helm delete mymab --purge